In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.seqsignet_functions import seqsignet_hyperparameter_search

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-18 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [11]:
features = ["timeline_index"]
standardise_method = [None]
num_features = len(features)
include_features_in_path = True

In [12]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]  # [50, 15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
lstm_hidden_dim_sizes = [384]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [13]:
shift = 3
window_size = 5
n = 6

In [14]:
size = 20
(
    seqsignet_network_umap_kfold_20,
    best_seqsignet_network_umap_kfold_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_20_kfold.csv


In [15]:
seqsignet_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.645594,0.484241,"[0.7791275538376587, 0.3548085901027078, 0.318...",0.495771,"[0.745377707342842, 0.40860215053763443, 0.333...",0.478355,"[0.8160786581839213, 0.31353135313531355, 0.30...",None,0.672768,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.639080,0.499637,"[0.7662780779073074, 0.39300937766410915, 0.33...",0.504367,"[0.7536353467561522, 0.40652557319223986, 0.35...",0.495663,"[0.7793522267206477, 0.3803630363036304, 0.327...",None,0.674534,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.643487,0.484309,"[0.7741846323935876, 0.3694083694083694, 0.309...",0.495638,"[0.7413975648491266, 0.4429065743944637, 0.302...",0.481067,"[0.8100057836899942, 0.31683168316831684, 0.31...",None,0.673892,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.642146,0.481508,"[0.7790032109451348, 0.364983788791107, 0.3005...",0.488287,"[0.7530364372469636, 0.4160506863780359, 0.295...",0.479121,"[0.8068247541931752, 0.3250825082508251, 0.305...",None,0.663616,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.638889,0.477189,"[0.7671935070482698, 0.42063804647499015, 0.24...",0.494770,"[0.755680224403927, 0.4024114544084401, 0.3262...",0.471401,"[0.7790630422209369, 0.4405940594059406, 0.194...",None,0.646435,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.637931,0.482511,"[0.7708098987626546, 0.378682842287695, 0.2980...",0.490755,"[0.7501368363437328, 0.3987226277372263, 0.323...",0.476526,"[0.7926547137073453, 0.36056105610561057, 0.27...",None,0.670841,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.626628,0.496121,"[0.7577823895641863, 0.4191063174114022, 0.311...",0.494160,"[0.7773722627737226, 0.3930635838150289, 0.312...",0.499637,"[0.7391555812608445, 0.44884488448844884, 0.31...",None,0.656230,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.626054,0.486406,"[0.7523986825146785, 0.39432048681541587, 0.31...",0.494599,"[0.7452482269503546, 0.38786911412609737, 0.35...",0.480832,"[0.7596876807403123, 0.400990099009901, 0.2818...",None,0.666506,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.629502,0.486077,"[0.7612535612535613, 0.38235294117647056, 0.31...",0.487122,"[0.7501403705783268, 0.41569767441860467, 0.29...",0.487675,"[0.772700983227299, 0.35396039603960394, 0.336...",None,0.667470,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [16]:
best_seqsignet_network_umap_kfold_20["f1"].mean()

0.5067445467413475

In [17]:
best_seqsignet_network_umap_kfold_20["precision"].mean()

0.5071407367751807

In [18]:
best_seqsignet_network_umap_kfold_20["recall"].mean()

0.5071599708650256

In [19]:
np.stack(best_seqsignet_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.76760324, 0.42523183, 0.32739857])

In [20]:
np.stack(best_seqsignet_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.76712029, 0.42856461, 0.32573731])

In [21]:
np.stack(best_seqsignet_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.76845961, 0.42271727, 0.33030303])

## GRP

In [ ]:
size = 20
(
    seqsignet_network_grp_kfold_20,
    best_seqsignet_network_grp_kfold_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_network_grp_kfold_20

In [ ]:
best_seqsignet_network_grp_kfold_20["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["recall_scores"]).mean(axis=0)